In [158]:
# !pip install MetaTrader5
import MetaTrader5 as mt5
import pandas as pd
# from tqdm.notebook import tqdm

if not mt5.initialize():
    mt5.shutdown()
mt5.terminal_info()

TerminalInfo(community_account=False, community_connection=False, connected=True, dlls_allowed=False, trade_allowed=True, tradeapi_disabled=False, email_enabled=False, ftp_enabled=False, notifications_enabled=False, mqid=False, build=2940, maxbars=100000, codepage=0, ping_last=161187, community_balance=0.0, retransmission=0.0, company='FX Choice Limited', name='FXChoice MetaTrader 5', language='English', path='C:\\Program Files\\FXChoice MetaTrader 5', ...)

In [110]:
risk = pd.read_csv('1y_risk.csv')
# risk

In [112]:
# get risk (size and SL dist)


In [114]:
# get risk * denomination * contract size

# denomination 
syms = set(list(risk['sym']) + ['GBPNZD'] + ['EURCAD'])
# syms

In [115]:
ticks = {}
for sym in syms:
    
    # print(sym)
    t = mt5.symbol_info_tick(sym)
    # print(t)
    p = (t.bid + t.ask) / 2
    ticks[sym] = p
    
#     if sym[3:] == 'USD':
#         print(sym, 1)
#     else:
#         print(sym, '1 / ' + sym[3:] + 'USD')

In [116]:
denom = {}
for sym in syms:
    if sym[3:] == 'USD':
        # print(sym, 1)
        denom[sym] = 1
    else:
        if sym[3:] + 'USD' in syms:
            # print(sym, sym[3:] + 'USD')
            denom[sym] = ticks[sym[3:] + 'USD']
        else:
            # print(sym, 'USD' + sym[3:], 1/ticks['USD' + sym[3:]])
            denom[sym] = 1/ticks['USD' + sym[3:]]

In [118]:
# denom

In [119]:
risk = risk.to_dict(orient='records')

In [134]:
# risk * denom * contract

scaled_mvmt = {}

for sym in syms:
    if sym in [_['sym'] for _ in risk]:
        r = list(filter(lambda x: x['sym'] == sym, risk))[0]['avg+std']
        if sym[3:] == 'JPY':
            r = r/100
        if sym[3:] != 'JPY' and sym != 'BTCUSD' and sym != 'XAUUSD':
            r = r/10000
        c = 100 if sym == 'XAUUSD' else 1 if sym == 'BTCUSD' else 100000
        scaled_mvmt[sym] = r * denom[sym] * c

# scaled_mvmt

In [127]:
# account balance
account = mt5.account_info()

# account.balance

btc = mt5.symbol_info_tick('BTCUSD')
btc_mid = (btc.bid + btc.ask) / 2
btc_mid

usd = account.balance * btc_mid * 0.000001

fr = 0.025

dr = usd * fr

In [136]:
size = {}
for sym in syms:
    if sym in [_['sym'] for _ in risk]:
        # print(sym, dr/scaled_mvmt[sym])
        size[sym] = round(dr/scaled_mvmt[sym], 2)

In [137]:
size

{'USDCAD': 0.1,
 'NZDJPY': 0.12,
 'EURAUD': 0.08,
 'GBPAUD': 0.06,
 'CADJPY': 0.13,
 'BTCUSD': 0.03,
 'AUDJPY': 0.1,
 'GBPCAD': 0.07,
 'USDCHF': 0.11,
 'NZDCHF': 0.12,
 'EURUSD': 0.09,
 'CHFJPY': 0.11,
 'GBPUSD': 0.06,
 'XAUUSD': 0.02,
 'USDJPY': 0.13,
 'EURNZD': 0.08,
 'GBPJPY': 0.07,
 'AUDUSD': 0.1,
 'EURJPY': 0.1,
 'AUDCHF': 0.11,
 'EURCHF': 0.13,
 'AUDCAD': 0.13,
 'GBPCHF': 0.07,
 'CADCHF': 0.13,
 'NZDUSD': 0.11,
 'EURGBP': 0.08}

In [149]:
dist = {}
for sym in syms:
    c = 100 if sym == 'XAUUSD' else 1 if sym == 'BTCUSD' else 1000 if sym[3:] == 'JPY' else 100000
    if sym in [_['sym'] for _ in risk]:
        # print(sym, scaled_mvmt[sym] / c)
        if sym[3:] == 'JPY':
            dist[sym] = round(scaled_mvmt[sym] / c, 3)
        else:
            dist[sym] = round(scaled_mvmt[sym] / c, 5)

In [150]:
dist

{'USDCAD': 0.00214,
 'NZDJPY': 0.186,
 'EURAUD': 0.00272,
 'GBPAUD': 0.00335,
 'CADJPY': 0.171,
 'BTCUSD': 861.8,
 'AUDJPY': 0.207,
 'GBPCAD': 0.00327,
 'USDCHF': 0.00197,
 'NZDCHF': 0.00182,
 'EURUSD': 0.00234,
 'CHFJPY': 0.197,
 'GBPUSD': 0.0035,
 'XAUUSD': 9.49,
 'USDJPY': 0.162,
 'EURNZD': 0.0027,
 'GBPJPY': 0.327,
 'AUDUSD': 0.00221,
 'EURJPY': 0.22,
 'AUDCHF': 0.00197,
 'EURCHF': 0.00163,
 'AUDCAD': 0.00163,
 'GBPCHF': 0.00327,
 'CADCHF': 0.00165,
 'NZDUSD': 0.002,
 'EURGBP': 0.0027}

In [154]:
risk_new = {}
for s in size.keys():
    risk_new[s] = {'size': size[s]}
for d in dist.keys():
    risk_new[d]['dist'] = dist[d]

In [155]:
risk_new

{'USDCAD': {'size': 0.1, 'dist': 0.00214},
 'NZDJPY': {'size': 0.12, 'dist': 0.186},
 'EURAUD': {'size': 0.08, 'dist': 0.00272},
 'GBPAUD': {'size': 0.06, 'dist': 0.00335},
 'CADJPY': {'size': 0.13, 'dist': 0.171},
 'BTCUSD': {'size': 0.03, 'dist': 861.8},
 'AUDJPY': {'size': 0.1, 'dist': 0.207},
 'GBPCAD': {'size': 0.07, 'dist': 0.00327},
 'USDCHF': {'size': 0.11, 'dist': 0.00197},
 'NZDCHF': {'size': 0.12, 'dist': 0.00182},
 'EURUSD': {'size': 0.09, 'dist': 0.00234},
 'CHFJPY': {'size': 0.11, 'dist': 0.197},
 'GBPUSD': {'size': 0.06, 'dist': 0.0035},
 'XAUUSD': {'size': 0.02, 'dist': 9.49},
 'USDJPY': {'size': 0.13, 'dist': 0.162},
 'EURNZD': {'size': 0.08, 'dist': 0.0027},
 'GBPJPY': {'size': 0.07, 'dist': 0.327},
 'AUDUSD': {'size': 0.1, 'dist': 0.00221},
 'EURJPY': {'size': 0.1, 'dist': 0.22},
 'AUDCHF': {'size': 0.11, 'dist': 0.00197},
 'EURCHF': {'size': 0.13, 'dist': 0.00163},
 'AUDCAD': {'size': 0.13, 'dist': 0.00163},
 'GBPCHF': {'size': 0.07, 'dist': 0.00327},
 'CADCHF': {'s

In [160]:
# get current working orders
o = mt5.orders_get()

In [163]:
o[0]

TradeOrder(ticket=5635941, time_setup=1621235479, time_setup_msc=1621235479802, time_done=0, time_done_msc=0, time_expiration=0, type=2, type_time=0, type_filling=2, state=1, magic=9, position_id=0, position_by_id=0, reason=3, volume_initial=0.08, volume_current=0.08, price_open=1.66144, sl=1.6602299999999999, tp=0.0, price_current=1.7111800000000001, price_stoplimit=0.0, symbol='EURNZD', comment='sent by python', external_id='')

In [162]:
o[0].ticket

5635941

In [169]:
# r = {
#     "action": mt5.TRADE_ACTION_MODIFY, # TRADE_ACTION_REMOVE
#     "order": 5635941,
#     # "symbol": _[0],
#     "volume": 0.09,
#     # "type": mt5.ORDER_TYPE_BUY_LIMIT,
#     "price": 1.66144,
#     "sl": 1.66,  ############ this is getting mixed up for shorts
#     # "tp": _[1] + (_[3]*3),
#     # "deviation": 10,
#     "magic": 9,
#     "comment": "sent by python",
#     # "type_time": mt5.ORDER_TIME_GTC, # good till cancelled
#     # "type_filling": mt5.ORDER_FILLING_RETURN,
# }
# result = mt5.order_send(r) 

In [197]:
def update_order(tradeorder):
    
    # get the new params
    print(tradeorder.symbol)
    
    # check if params different
    print(risk_new[tradeorder.symbol])
    
    # will fail for sells
    current_dist = round(abs(tradeorder.price_open - tradeorder.sl), 4)
    print('current dist:', current_dist)
    # if different, update to new
    
    # if no changes, return
    if risk_new[tradeorder.symbol]['dist'] != current_dist and risk_new[tradeorder.symbol]['size'] != tradeorder.volume_current:
        return None
    
    # set new_vol to current in case no change
    new_vol = tradeorder.volume_current
    # set new_sl to current in case no change
    new_sl = tradeorder.sl
    
    if risk_new[tradeorder.symbol]['dist'] != current_dist:
        print('  change dist')
        if tradeorder.type == 2:
            print('    new buy sl')
            new_sl = tradeorder.price_open - risk_new[tradeorder.symbol]['dist']
            print('    ', new_sl)
            
            o_type = mt5.ORDER_TYPE_BUY_LIMIT
            
        elif tradeorder.type == 3:
            print('    new sell sl')
            new_sl = risk_new[tradeorder.symbol]['dist'] - tradeorder.price_open
            print('    ', new_sl)
            
            o_type = mt5.ORDER_TYPE_SELL_LIMIT
        
    if risk_new[tradeorder.symbol]['size'] != tradeorder.volume_current:
        print('  change volume')
        new_vol = risk_new[tradeorder.symbol]['size']
        print('    ', new_vol)
    
    # cancel order
    r = {
        "action": mt5.TRADE_ACTION_REMOVE,
        "order": tradeorder.ticket
    }
    result = mt5.order_send(r) 
    
    print(result)
    
    r = {
        "action": mt5.TRADE_ACTION_PENDING,
        "symbol": tradeorder.symbol,
        "volume": new_vol,
        "type": o_type,
        "price": tradeorder.price_open,
        "sl": new_sl,
        # "tp": _[1] + (_[3]*3),
        "deviation": 10,
        "magic": 9,
        "comment": "sent by python",
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_RETURN,
    }
    result = mt5.order_send(r) 
    print(result)
    
    return result

In [198]:
update_order(o[0])

EURNZD
{'size': 0.08, 'dist': 0.0027}
current dist: 0.0012
  change dist
    new buy sl
     1.65874
OrderSendResult(retcode=10009, deal=0, order=5635941, volume=0.0, price=0.0, bid=0.0, ask=0.0, comment='Request executed', request_id=9, retcode_external=0, request=TradeRequest(action=8, magic=0, order=5635941, symbol='', volume=0.0, price=0.0, stoplimit=0.0, sl=0.0, tp=0.0, deviation=0, type=0, type_filling=0, type_time=0, expiration=0, comment='', position=0, position_by=0))
OrderSendResult(retcode=10009, deal=0, order=5785562, volume=0.08, price=0.0, bid=0.0, ask=0.0, comment='Request executed', request_id=10, retcode_external=0, request=TradeRequest(action=5, magic=9, order=0, symbol='EURNZD', volume=0.08, price=1.66144, stoplimit=0.0, sl=1.65874, tp=0.0, deviation=10, type=2, type_filling=2, type_time=0, expiration=0, comment='sent by python', position=0, position_by=0))


OrderSendResult(retcode=10009, deal=0, order=5785562, volume=0.08, price=0.0, bid=0.0, ask=0.0, comment='Request executed', request_id=10, retcode_external=0, request=TradeRequest(action=5, magic=9, order=0, symbol='EURNZD', volume=0.08, price=1.66144, stoplimit=0.0, sl=1.65874, tp=0.0, deviation=10, type=2, type_filling=2, type_time=0, expiration=0, comment='sent by python', position=0, position_by=0))